In [1]:
import networkx as nx
from scipy.linalg import expm
import numpy as np
from itertools import count
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import osmnx as ox
import matplotlib.patches as mpatches

In [ ]:
group1 = ['motorway', 'motorway_link', 'trunk', 'trunk_link']
group2 = ['primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary', 'tertiary_link']
group3 = group1 + group2

city_list_full = ["São Paulo", # 0
                  "Rio de Janeiro", # 1
                  "Atlanta", # 2
                  "Manhattan", # 3
                  "Barcelona", # 4
                  "Madrid", # 5
                  "Buenos Aires", # 6 
                  "London", # 7
                  "Beijing", # 8
                  "Paris", # 9
                  "Cardiff", # 10
                  "Berlin", # 11
                  "Amsterdam", # 12
                  "São José dos Campos", # 13
                  "Los Angeles", # 14
                  "Toulouse", # 15
                  "Wichita", # 16
                  "Salt Lake", # 17
                  ]

lat_long_list_full = [(-23.546, -46.634), # 0
                      (-22.91100731359119, -43.20939643363555), # 1
                      (33.755, -84.388), # 2
                      (40.748, -73.985), # 3
                      (41.390, 2.166), # 4
                      (40.416, -3.703), # 5
                      (-34.609532137872215, -58.413562281598004), # 6
                      (51.51350693217751, -0.10620383552655813), # 7
                      (39.91410059119869, 116.39035428390193), # 8
                      (48.85968272013924, 2.343524958942639), # 9
                      (51.47684668368528, -3.1781309848863764), # 10
                      (52.51753698669687, 13.412986656538179), # 11
                      (52.37331117387958, 4.901974990038526), # 12
                      (-23.184974237895485, -45.87883533129896), # 13
                      (34.04325810825251, -118.24378505331654), # 14
                      (43.604495768999215, 1.4474979660394318), # 15
                      (37.6858506794406, -97.33237822581114), # 16
                      (40.7610515545281, -111.89953708354962), # 17
                      ]

dist = 4000
num = 0
for latlong, city in zip(lat_long_list_full[num:], city_list_full[num:]):
    print(city, num)
    G = ox.graph_from_point(latlong, dist = dist, truncate_by_edge = True, network_type="drive")

    comps = [].extend(nx.strongly_connected_components(G))
    G = G.subgraph(max(nx.strongly_connected_components(G), key = len))

    A = nx.to_numpy_array(G, dtype = float)
    np.fill_diagonal(A, 0)
    A = np.maximum(A, A.T)

    comm = expm(A)
    df_comm = pd.DataFrame(data = comm, columns = G.nodes(), index = G.nodes())
    nodelist = G.nodes
    diag = np.diag(df_comm)
    nodelist = list(nodelist)
    size = len(nodelist)
    cd = np.zeros((size,size))

    for i in range(size):
        for j in range(size):
            cd[i][j] = np.sqrt(diag[i] + diag[j] - 2*comm[i][j])

    cd = pd.DataFrame(data = cd, index = nodelist, columns = nodelist)

    dfA = nx.to_pandas_adjacency(G)
    X = (dfA*cd)
    GX = nx.from_pandas_adjacency(X, nx.DiGraph)

    j = dict()
    for i in list(G.edges):
        try:
            j[i[0], i[1], 0] =  nx.dijkstra_path_length(G, i[1], i[0],  weight = "length")
        except:
            j[i[0], i[1], 0] = 0
    nx.set_edge_attributes(G, j, "Inverse SP")

    j = dict(nx.all_pairs_dijkstra_path_length(GX, weight = "weight"))
    dfSP = nx.to_pandas_adjacency(G)

    teste = pd.DataFrame.from_dict(j, orient = "index")
    for i in teste.columns:
        for j in teste.columns:
            if np.isnan(teste[i][j]):
                dfSP[i][j] = 0
            else:
                dfSP[i][j] = teste[i][j]

    ew_flow = dict()

    for i in G.edges:
      ew_flow[(i[0], i[1], 0)] = dfSP[i[0]][i[1]]

    nx.set_edge_attributes(G, ew_flow, "Cost of return")
    nx.set_edge_attributes(G, nx.edge_betweenness_centrality(G, weight = "length"), "Edge Betweenness")


    nodes, edges = ox.graph_to_gdfs(G)
    nodes.to_csv(f"Results/csv/{city}_nodes.csv")
    edges["highway"] = edges.highway.map(lambda x: x[0] if isinstance(x, list) else x)
    edges["Groups"] = edges.highway
    edges["Groups"] = edges.Groups.map(lambda x: 'A' if x  in group1 else x)
    edges["Groups"] = edges.Groups.map(lambda x: 'B' if x  in group2 else x)
    edges["Groups"] = edges.Groups.map(lambda x: 'C' if x  not in group3 else x)
    num+=1
    edges.to_csv(f"Results/csv/{city}_edges.csv")

Toulouse 7


C:\Users\gio-x\AppData\Local\Temp\ipykernel_16216\3486306374.py:92: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dfSP[i][j] = teste[i][j]
